## Introduction

Bandung is the capital city of West Java - Indonesia. In the early 20th century the Dutch inhabitants of Bandung demanded the establishment of a municipality (gemeente), which was granted in 1906, and Bandung gradually developed into a resort city for plantation owners. Luxurious hotels, restaurants, cafés, and European boutiques were opened, leading the city to be nicknamed Parijs van Java (Dutch: "The Paris of Java") (Ref. Wikipedia).<br>
Bandung tourism industry increases exponentially as the Airport accomodates more international flights. This attracts investors locally and internationally to establish their business in Bandung. However, due to language barrier and lack of information of tourist destination, foreigner who wish to invest may find it difficult to understand the regulation and best location before they decide to open a business.<br>
This report is created for potential investors to determine the best location for their business based on demographic data of neighborhood in Bandung.

## Data

To perform this analysis, we will need the following data:<br>
 - List of the districts of Budapest
 - Geo-coordinates of the districts in Budapest
 - Top venues of districts

List of districts will be obtained from Wikipedia. (https://id.wikipedia.org/wiki/Daftar_kecamatan_dan_kelurahan_di_Kota_Bandung)<br>

Geo-coordinates of districts will be obtained from the local government database (http://data.bandung.go.id/dataset/1fb076f0-24f8-4abe-b418-3b62083e15e8/resource/a6a429b4-18ff-4973-8310-c99ed4db6e08/download/koordinat-dan-ketinggian-kantor-kecamatan-di-kota-bandung-2014.csv).<br>

Top venues data will be obtained from Foursquare through an API.

In [2]:
!pip install bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [3]:
source = requests.get('https://id.wikipedia.org/wiki/Daftar_kecamatan_dan_kelurahan_di_Kota_Bandung').text
soup=BeautifulSoup(source, 'html5lib')
tables = soup.find_all('table')
df = pd.read_html(str(tables[0]), flavor='bs4')[0]

df

,Kode Kemendagri,Kecamatan,Jumlah Kelurahan,Daftar Kelurahan
0,32.73.05,Andir,6,Campaka Ciroyom Dunguscariang Garuda Kebonjeru...
1,32.73.10,Astana Anyar,6,Cibadak Karanganyar Karasak Nyengseret Panjuna...
2,32.73.20,Antapani,4,Antapani Kidul Antapani Kulon Antapani Tengah ...
3,32.73.24,Arcamanik,4,Cisaranten Bina Harapan Cisaranten Endah Cisar...
4,32.73.03,Babakan Ciparay,6,Babakan Babakanciparay Cirangrang Margahayu Ut...
5,32.73.21,Bandung Kidul,4,Batununggal Kujangsari Mengger Wates
6,32.73.15,Bandung Kulon,8,Caringin Cibuntu Cigondewah Kaler Cigondewah K...
7,32.73.09,Bandung Wetan,3,Cihapit Citarum Tamansari
8,32.73.12,Batununggal,8,Binong Cibangkong Gumuruh Kacapiring Kebongeda...
9,32.73.04,Bojongloa Kaler,5,Babakan Asih Babakan Tarogong Jamika Kopo Suka...


In [4]:
lat_lon = pd.read_csv('http://data.bandung.go.id/dataset/1fb076f0-24f8-4abe-b418-3b62083e15e8/resource/a6a429b4-18ff-4973-8310-c99ed4db6e08/download/koordinat-dan-ketinggian-kantor-kecamatan-di-kota-bandung-2014.csv')
lat_lon.head()

,Kecamatan,Lintang Selatan,Bujur Timur,Ketinggian (dpl)
0,Bandung Kulon,6.92520,107.5745,709
1,Babakan Ciparay,6.93710,107.5830,697
2,Bojongloa Kaler,6.93880,107.5938,694
3,Bojongloa Kidul,6.94285,107.5967,689
4,Astanaanyar,6.93190,107.5998,695
